**Read stuff from your own gdrive**

In [0]:
MY_HOME = 'drive/My Drive/'


In [0]:
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.metrics import classification_report

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

seed = 123
np.random.seed(seed)

def to_tdidf(pickle_path="tdidf_method.pkl"):
    print("==========> Data Preprocessing")
    data = pd.read_pickle(os.path.join(MY_HOME, "preprocessed_training_dataframe.pkl"))

    print(data.head())
    lim_unigram = 5000
    # no more ram, limit this
    vectorizer = TfidfVectorizer(max_features=lim_unigram)
    vectorizer.fit(data['X'][0:15555])

    # fp = os.path.join(MY_HOME, pickle_path)
    # with open(fp, 'wb') as jar:
    #     pickle.dump(vectorizer, jar, protocol=pickle.HIGHEST_PROTOCOL)
    
    return vectorizer

def fc(vectorize, denselist, data):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50), random_state=1)
    # TODO: deinfe amx feat, feat on train and test, tranfsofrm on individuals
    for i in range(14):#range(data.shape[0]//10):
        train_data = vectorize.transform(data['X'][i:i+1000])

        clf.fit(train_data, data['label'][i:i+1000])
    X_test = vectorize.transform(data['X'][15000:15555])
    y_test = data['label'][15000:15555]
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

if __name__ == '__main__':
    # vectorizer = to_tdidf()
    # with open(os.path.join(MY_HOME,"tdidf_list.pkl"), 'rb') as jar:
    #     dense_arr = pickle.load(jar)

    # print(dense_arr)
    # data = pd.read_pickle(os.path.join(MY_HOME, "preprocessed_training_dataframe.pkl"))
    # print(data.shape)
    # fc(vectorizer, dense_arr, data)

==========> Data Preprocessing
    label                                                  X
id                                                          
0       0   a lin from georg orwel 's novel 1984 predict ...
1       2   main legisl candid les gibson insult parkland...
4       1   a 17-year-old girl nam alyss carson be be tra...
5       2   in 1988 auth roald dahl pen an op let urg par...
6       2  hil clinton when it com to fight ter , `` anot...
(15555, 2)
              precision    recall  f1-score   support

           0       0.59      0.70      0.64       261
           1       0.52      0.50      0.51       231
           2       0.09      0.03      0.05        63

    accuracy                           0.54       555
   macro avg       0.40      0.41      0.40       555
weighted avg       0.50      0.54      0.52       555



In [0]:
!pip install PyDrive



In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:

# Import the corpus data

from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#!ls drive/My\ Drive
MY_HOME = 'drive/My\ Drive/' #metadata_articles_dataframe.pkl
!ls drive

'My Drive'


# KERAS to try the word embedding method

In [120]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

data = pd.read_pickle(os.path.join(MY_HOME, "nlp/preprocessed_training_dataframe.pkl"))
# define documents
docs = data['X'][0:12000]
# define class labels
labels = data['label'][0:12000]
print(labels)
# convert labels to onehot
labels = one_hot(labels, 3)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 4 words
max_length = 300
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# load the whole embedding into memory
embeddings_index = dict()
f = open(os.path.join(MY_HOME, 'nlp/glove.6B.50d.txt'))
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())

model.add(Dense(3, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=2, verbose=0)


id
0        0
1        2
4        1
5        2
6        2
        ..
13211    2
13212    0
13213    1
13214    1
13215    0
Name: label, Length: 12000, dtype: int64
Loaded 400000 word vectors.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 300, 50)           16058550  
_________________________________________________________________
flatten_7 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 45003     
Total params: 16,103,553
Trainable params: 45,003
Non-trainable params: 16,058,550
_________________________________________________________________
None


In [121]:
# evaluate the model
# define documents
docs = data['X'][12000:15555]
# define class labels
labels = data['label'][12000:15555]
print(labels)
# convert labels to onehot
labels = one_hot(labels, 3)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 4 words
max_length = 300
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

id
13216    1
13217    0
13218    1
13219    2
13220    0
        ..
17137    2
17138    0
17139    0
17140    2
17141    1
Name: label, Length: 3555, dtype: int64
Accuracy: 62.512894


In [0]:
!pip install torch<=1.2.0
!pip install torchtext==0.4
%matplotlib inline

/bin/bash: =1.2.0: No such file or directory
     |████████████████████████████████| 61kB 2.8MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
